In [21]:
import numpy as np
import os
import stat

BATCH_DIR = '/home/ci411/volume_estimation/batched_jobs'

In [22]:
train_script = '''
import sys
sys.path.append("/home/ci411/volume_estimation/")

import model_funcs
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

MODELS_DIR = '/scratch/ci411/sonos_rirs/models/'
FEATURES_DIR = '/scratch/ci411/sonos_rirs/features/'

feature_set = '{feature_set}'

model_dict = {{}}
model_dict['name'] = '{model_name}'
model_dict['notes'] = '{model_notes}'
model_dict['data_path'] = os.path.join(FEATURES_DIR, feature_set, 'feature_df.csv')
model_dict['model_path'] = os.path.join(MODELS_DIR, '{experiment_name}', model_dict['name'])

model_funcs.train_model(model_funcs.Baseline_Model, model_dict, epochs={epochs}, batch_size={batch_size}, lr_init={lr_init}, l2_reg={l2_reg}, overwrite=True, log={log}, sched_thres={sched_thres}, target='{target}')
'''

#keys are feature_set, model_name, model_notes
#batch_size, lr_init, l2_reg, log, sched_thres

train_script_example = train_script.format(feature_set="cool_features",model_name="cool_model", model_notes="testing the formatting",\
                                           experiment_name='cool_experiment',\
                                           epochs=3000, batch_size=64, lr_init=1e-4, l2_reg=1e-4, log=False, sched_thres=1e-3, target="whatever")
print(train_script_example)


import sys
sys.path.append("/home/ci411/volume_estimation/")

import model_funcs
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

MODELS_DIR = '/scratch/ci411/sonos_rirs/models/'
FEATURES_DIR = '/scratch/ci411/sonos_rirs/features/'

feature_set = 'cool_features'

model_dict = {}
model_dict['name'] = 'cool_model'
model_dict['notes'] = 'testing the formatting'
model_dict['data_path'] = os.path.join(FEATURES_DIR, feature_set, 'feature_df.csv')
model_dict['model_path'] = os.path.join(MODELS_DIR, 'cool_experiment', model_dict['name'])

model_funcs.train_model(model_funcs.Baseline_Model, model_dict, epochs=3000, batch_size=64, lr_init=0.0001, l2_reg=0.0001, overwrite=True, log=False, sched_thres=0.001, target='whatever')



In [23]:
batch_script = '''#!/bin/bash

#SBATCH --job-name={job_name}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=32GB
#SBATCH --time=47:59:59
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --export=NONE
#SBATCH --output="o_{job_name}-%j.out"
#SBATCH --gres=gpu:{gre_spec}

module purge
module load anaconda3/2020.07

source /home/ci411/.bashrc
conda activate s3d_env
python /home/ci411/volume_estimation/batched_jobs/{experiment_name}/{job_name}.py
'''

test_batch = batch_script.format(job_name="jobbyjob", experiment_name="science_time", gre_spec='v100:1')
print(test_batch)

#!/bin/bash

#SBATCH --job-name=jobbyjob
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=32GB
#SBATCH --time=47:59:59
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --export=NONE
#SBATCH --output="o_jobbyjob-%j.out"
#SBATCH --gres=gpu:v100:1

module purge
module load anaconda3/2020.07

source /home/ci411/.bashrc
conda activate s3d_env
python /home/ci411/volume_estimation/batched_jobs/science_time/jobbyjob.py



In [17]:
#set default hyperparameters
def_hyp = {}
def_hyp['epochs'] = 1000
def_hyp['batch_size'] = 128
def_hyp['lr_init'] = 1.85e-5
def_hyp['l2_reg'] = 1.67e-3
def_hyp['log'] = True
def_hyp['sched_thres'] = .1
def_hyp['target'] = 'rt60'
def_hyp['gre_spec'] = 'v100:1'


#comparison models
experiment_name = '081922_rt60_comp'
run_template = 'prop{}_rt60_081922'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

indexes = [0,1,2,3,4,5,6]
feature_set_template = '081522_20k/081822_20k_prop{}'
runall = ""

notes_list = ['baseline: mag + mag-feats',\
              'best model: mag + lf phase + lf 1st deriv',\
              'full phase1: fb phase + mag-feats',\
              'full phase2: fb phase',\
              'replace-cont: mag + lf phase + lf cont',\
              'base-cont: mag + mag-feats + lf cont',\
              'best-cont: mag + lf phase + lf 1st deriv + lf cont']

for i, idx in enumerate(indexes):
    job_name = run_template.format(i)
    notes = notes_list[i]
    feature_set = feature_set_template.format(idx)
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=def_hyp['epochs'],\
                                batch_size=def_hyp['batch_size'],lr_init=def_hyp['lr_init'],\
                                l2_reg=def_hyp['l2_reg'], log=def_hyp['log'],\
                                sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target='rt60')
    
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec=def_hyp['gre_spec'])
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [17]:
#set default hyperparameters
def_hyp = {}
def_hyp['epochs'] = 1000
def_hyp['batch_size'] = 128
def_hyp['lr_init'] = 2e-5
def_hyp['l2_reg'] = 1.67e-3
def_hyp['log'] = True
def_hyp['sched_thres'] = .01
def_hyp['target'] = 'rt60'
def_hyp['gre_spec'] = 'v100:1'


#comparison models
experiment_name = '081922_rt60_comp'
run_template = 'prop{}_rt60_081922'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

indexes = [0,1,2,3,4,5,6]
feature_set_template = '081522_20k/081822_20k_prop{}'
runall = ""

notes_list = ['baseline: mag + mag-feats',\
              'best model: mag + lf phase + lf 1st deriv',\
              'full phase1: fb phase + mag-feats',\
              'full phase2: fb phase',\
              'replace-cont: mag + lf phase + lf cont',\
              'base-cont: mag + mag-feats + lf cont',\
              'best-cont: mag + lf phase + lf 1st deriv + lf cont']

for i, idx in enumerate(indexes):
    job_name = run_template.format(i)
    notes = notes_list[i]
    feature_set = feature_set_template.format(idx)
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=def_hyp['epochs'],\
                                batch_size=def_hyp['batch_size'],lr_init=def_hyp['lr_init'],\
                                l2_reg=def_hyp['l2_reg'], log=def_hyp['log'],\
                                sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target='rt60')
    
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec=def_hyp['gre_spec'])
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [33]:
#set default hyperparameters
def_hyp = {}
def_hyp['epochs'] = 1000
def_hyp['batch_size'] = 128
def_hyp['lr_init'] = 1.85e-5
def_hyp['l2_reg'] = 1.67e-3
def_hyp['log'] = True
def_hyp['sched_thres'] = .01
def_hyp['target'] = 'vol'
def_hyp['gre_spec'] = 'v100:1'

#comparison models
experiment_name = '081822_comp'
run_template = 'prop{}_20k_081822'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

indexes = [0,1,2,3,4,5,6]
feature_set_template = '081522_20k/081822_20k_prop{}'
runall = ""

notes_list = ['baseline: mag + mag-feats',\
              'best model: mag + lf phase + lf 1st deriv',\
              'full phase1: fb phase + mag-feats',\
              'full phase2: fb phase',\
              'replace-cont: mag + lf phase + lf cont',\
              'base-cont: mag + mag-feats + lf cont',\
              'best-cont: mag + lf phase + lf 1st deriv + lf cont']

for i, idx in enumerate(indexes):
    job_name = run_template.format(i)
    notes = notes_list[i]
    feature_set = feature_set_template.format(idx)
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=def_hyp['epochs'],\
                                batch_size=def_hyp['batch_size'],lr_init=def_hyp['lr_init'],\
                                l2_reg=def_hyp['l2_reg'], log=def_hyp['log'],\
                                sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec=def_hyp['gre_spec'])
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [34]:
#set default hyperparameters
def_hyp = {}
def_hyp['epochs'] = 1000
def_hyp['batch_size'] = 128
def_hyp['lr_init'] = 1.85e-5
def_hyp['l2_reg'] = 1.67e-3
def_hyp['log'] = True
def_hyp['sched_thres'] = .01
def_hyp['target'] = 'vol'
def_hyp['gre_spec'] = 'v100:1'

#comparison models
experiment_name = '081822_2_comp'
run_template = 'prop{}_20k_081522'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

indexes = [0,1,2,3,4,5,6]
feature_set_template = '081522_20k/081522_20k_prop{}'
runall = ""

notes_list = ['baseline: mag + mag-feats',\
              'best model: mag + lf phase + lf 1st deriv',\
              'ablation 1: lf phase + lf 1st deriv',\
              'ablation 2: mag + lf 1st deriv',\
              'ablation 3: mag + lf phase',\
              'base-cont: mag + mag-feats + lf cont',\
              'best-cont: mag + lf phase + lf 1st deriv + lf cont']

for i, idx in enumerate(indexes):
    job_name = run_template.format(i)
    notes = notes_list[i]
    feature_set = feature_set_template.format(idx)
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=def_hyp['epochs'],\
                                batch_size=def_hyp['batch_size'],lr_init=def_hyp['lr_init'],\
                                l2_reg=def_hyp['l2_reg'], log=def_hyp['log'],\
                                sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec=def_hyp['gre_spec'])
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [28]:
#set default hyperparameters
def_hyp = {}
def_hyp['epochs'] = 1000
def_hyp['batch_size'] = 128
def_hyp['lr_init'] = 2e-5
def_hyp['l2_reg'] = 1.67e-3
def_hyp['log'] = True
def_hyp['sched_thres'] = .01
def_hyp['target'] = 'rt60'
def_hyp['gre_spec'] = 'v100:1'

In [29]:
#vary batch size on baseline model
experiment_name = '082222_batch_size'
run_template = 'bs_0822{}'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

batch_sizes = [16,32,64,128,256,512,1028]
feature_set = '081522_20k/081822_20k_prop1'
runall = ""

for i, bs in enumerate(batch_sizes):
    job_name = run_template.format(i)
    notes = "varying batch size on baseline w/ batch_size {}".format(bs)
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=100, batch_size=bs,\
                                lr_init=def_hyp['lr_init'], l2_reg=def_hyp['l2_reg'],\
                                log=def_hyp['log'], sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec='1')
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [30]:
#vary inital learning rate on baseline model
experiment_name = '082222_learning_rate'
run_template = 'lr_0822{}'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

feature_set = '081522_20k/081822_20k_prop1'
runall = ""
    
#set variations
learning_rates = np.logspace(-1, -8, 8)

#adjust iteration
for i, lr in enumerate(learning_rates):
    job_name = run_template.format(i)
    #rewrite notes
    notes = "varying initial learning rate w/ lr = {}".format(lr)
    #adjust parameter inputs
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=300, batch_size=def_hyp['batch_size'],\
                                lr_init=lr, l2_reg=def_hyp['l2_reg'],\
                                log=def_hyp['log'], sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    #clear down
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec='1')
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [31]:
#vary l2 reg on baseline model
experiment_name = '082222_l2_reg'
run_template = 'l2_0822{}'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

feature_set = '081522_20k/081822_20k_prop1'
runall = ""
    
#set variations
reg_lambdas = np.logspace(-1, -5, 6)

#adjust iteration
for i, l2 in enumerate(reg_lambdas):
    job_name = run_template.format(i)
    #rewrite notes
    notes = "varying l2 reg parameter w/ lambda {}".format(l2)
    #adjust parameter inputs
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=500, batch_size=def_hyp['batch_size'],\
                                lr_init=def_hyp['lr_init'], l2_reg=l2,\
                                log=def_hyp['log'], sched_thres=def_hyp['sched_thres'],\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    #clear down
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec='1')
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [32]:
#vary scheduler threshold on baseline model
experiment_name = '082222_sched_thres'
run_template = 'st_0822{}'

experiment_dir = os.path.join(BATCH_DIR, experiment_name)

if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

feature_set = '081522_20k/081822_20k_prop1'
runall = ""
    
#set variations
thresholds = np.logspace(-3,-8, 5)

#adjust iteration
for i, th in enumerate(thresholds):
    job_name = run_template.format(i)
    #rewrite notes
    notes = "varying scheduler thresholds with threshold {}".format(th)
    #adjust parameter inputs
    train = train_script.format(feature_set=feature_set, model_name=job_name,\
                                model_notes=notes, epochs=500, batch_size=def_hyp['batch_size'],\
                                lr_init=def_hyp['lr_init'], l2_reg=def_hyp['l2_reg'],\
                                log=def_hyp['log'], sched_thres=th,\
                                experiment_name=experiment_name, target=def_hyp['target'])
    
    #clear down
    batch = batch_script.format(job_name=job_name, experiment_name=experiment_name, gre_spec='1')
    
    train_path = os.path.join(experiment_dir, job_name + '.py')
    batch_path = os.path.join(experiment_dir, job_name + '.sbatch')
    runall_path = os.path.join(experiment_dir, 'runall.bat')
    
    runall += "sbatch {}\n".format(batch_path)
    
    with open(train_path, 'w') as f:
        f.write(train)
        
    with open(batch_path, 'w') as f:
        f.write(batch)
    
    with open(runall_path, 'w') as f:
        f.write(runall)
        
    st = os.stat(runall_path)
    os.chmod(runall_path, st.st_mode | stat.S_IEXEC)

In [26]:
thresholds = np.logspace(-3,-8, 5)
thresholds

array([1.00000000e-03, 5.62341325e-05, 3.16227766e-06, 1.77827941e-07,
       1.00000000e-08])